In [1]:
import pandas as pd
import os

data_dir = "../../basins/"

country_match = {'Indonesia': 'ID', 'Bolivia': 'BO','Morocco': 'MA', 'Bangladesh': 'BD',
                 'Sri Lanka': 'LK', 'Kenya': 'KE', 'Mozambique': 'MZ',
                 "Afghanistan": "AF", "Uganda": "UG", "Rwanda": "RW", "Pakistan": "PK", 
                 "El Salvador": "SV", "Philippines": "PH", "Egypt": "EG", "Zambia": "ZM", "Angola": "AO",
                 "Ghana": "GH", "Côte d'Ivoire": "CI", "Honduras": "HN", "Kyrgyzstan": "KG"}

red_cols = ["workplaces_percent_change_from_baseline", 
            "retail_and_recreation_percent_change_from_baseline",
            "transit_stations_percent_change_from_baseline"]


def add_week_of_year(date): 
    if date.isocalendar()[1] < 10:    
        year_week = str(date.isocalendar()[0]) + "-0" + str(date.isocalendar()[1])
    else: 
        year_week = str(date.isocalendar()[0]) + "-" + str(date.isocalendar()[1])
    return year_week


def import_df(country):
    # import and loc
    try:
        df20 = pd.read_csv("./Region_Mobility_Report_CSVs/2020_" + country_match[country] + "_Region_Mobility_Report.csv")
        df21 = pd.read_csv("./Region_Mobility_Report_CSVs/2021_" + country_match[country] + "_Region_Mobility_Report.csv")
    except:
        print("Country not found:", country)
        return pd.DataFrame()

    if country == "Myanmar":
        country_r = "Myanmar (Burma)"
    else:
        country_r = country
    df = pd.concat((df20, df21), ignore_index=True)
    df = df.loc[(df.country_region == country_r) & (df.sub_region_1.isnull()) & 
                (df.sub_region_2.isnull()) & (df.metro_area.isnull())]
    
    # time steps
    df.reset_index(inplace=True, drop=True)
    df.date = pd.to_datetime(df.date)
    df.sort_values(by="date", inplace=True)
    df["year_week"] = df["date"].apply(add_week_of_year)
    
    # fix work columns
    df.loc[df.date.dt.weekday==5, "workplaces_percent_change_from_baseline"] = df["workplaces_percent_change_from_baseline"].shift(1)
    df.loc[df.date.dt.weekday==6, "workplaces_percent_change_from_baseline"] = df["workplaces_percent_change_from_baseline"].shift(1)

    return df


def get_reductions(df, columns):
    return (1 + df[columns].mean(axis=1) / 100)**2  
    

def save_reductions(df, country):
    df[["year_week", "red"]].groupby(by="year_week")["red"].mean().to_csv(os.path.join(data_dir, country, "restrictions/reductions.csv"))
    
    
def main(country, columns):
    df = import_df(country)
    if not df.empty:
        df["red"] = get_reductions(df, columns)
        save_reductions(df, country)

In [3]:
for country in country_match.keys():
    main(country, red_cols)